# Google Calendar → DataFrame (con `colorId`) 

Notebook para extraer eventos de **1 o más calendarios** usando la **Google Calendar API**, con `colorId` y **conteos por día/semana/mes**. Pensado para correr en local.

## Instrucciones rápidas
1. En **Google Cloud Console** crea un *OAuth Client ID (Desktop)* y descarga `credentials.json`.
2. Pon `credentials.json` en la misma carpeta donde ejecutes este notebook.
3. Ejecuta las celdas en orden. La primera vez se abrirá una ventana para autorizar el acceso.


## 2) Imports y configuración base

In [ ]:
from __future__ import annotations
import datetime as dt
from typing import List, Dict, Any
from tqdm.notebook import tqdm
import pandas as pd
import pytz
import re
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import calendar
import numpy as np
import os

SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']


TZ_DEFAULT = 'America/Santiago'  # cambia si lo necesitas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## 3) Autenticación y construcción del servicio

In [ ]:

def get_service(credentials_path: str = 'credentials.json', token_path: str = 'token.json'):
    creds = None

    # Try to load existing token
    if os.path.exists(token_path):
        try:
            creds = Credentials.from_authorized_user_file(token_path, SCOPES)
        except Exception:
            # Corrupt or incompatible token file – delete it and start fresh
            try:
                os.remove(token_path)
            except OSError:
                pass
            creds = None

    # If there are no (valid) creds, try to refresh; if that fails, re-auth
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except RefreshError:
                # Refresh token revoked/expired → delete token and force new consent
                try:
                    os.remove(token_path)
                except OSError:
                    pass
                creds = None

        if not creds or not creds.valid:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
            # Opción A (simple): sin include_granted_scopes
            creds = flow.run_local_server(
                port=0,
                access_type='offline',
                prompt='consent'
            )
        # Persist the fresh token
        with open(token_path, 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

service = get_service()

## 4) Listar calendarios disponibles

In [ ]:
def list_calendars(service) -> List[Dict[str, Any]]:
    cals = []
    page_token = None
    while True:
        res = service.calendarList().list(pageToken=page_token, maxResults=250).execute()
        cals.extend(res.get('items', []))
        page_token = res.get('nextPageToken')
        if not page_token:
            break
    return cals

calendars = list_calendars(service)
df_cals = pd.DataFrame([{'id': c['id'], 'summary': c.get('summary', ''), 'primary': c.get('primary', False)} for c in calendars])
df_cals

👉 Copia aquí los **IDs de tus 3 calendarios** (puedes usar `'primary'` para el principal).

In [ ]:
# EDITA ESTA LISTA con tus 3 calendarios
CAL_IDS = [
    'primary',
    '4umck5jddsi4iv1tlbprfg2ko0@group.calendar.google.com',
    'k96ls1bdk7te68hsbnlhs0bbt0@group.calendar.google.com',
]
len(CAL_IDS), CAL_IDS

## 5) Rango de fechas y zona horaria

In [ ]:
# EDITA las fechas a tu necesidad
START_DATE = '2000-01-01'  # inclusive
END_DATE   = '2026-08-31'  # inclusive

def iso_z(dt_str: str) -> str:
    return dt_str if dt_str.endswith('Z') else dt_str + 'Z'

start_iso = iso_z(START_DATE + 'T00:00:00')
end_dt = dt.datetime.fromisoformat(END_DATE) + dt.timedelta(days=1)
end_iso = iso_z(end_dt.strftime('%Y-%m-%dT00:00:00'))

start_iso, end_iso


## 6) Descargar mapa de colores (event y calendar)

In [ ]:
def get_colors_map(service) -> Dict[str, Dict[str, str]]:
    colors = service.colors().get().execute()
    return {
        "event": {k: v.get('background', '') for k, v in colors.get('event', {}).items()},
        "calendar": {k: v.get('background', '') for k, v in colors.get('calendar', {}).items()},
    }

colors_map = get_colors_map(service)
colors_map['event']  # colorId -> HEX


## 7) Funciones utilitarias (fetch de eventos y fecha local)

In [ ]:
def fetch_events(service, calendar_id: str, time_min: str, time_max: str, max_results: int = 2500):
    events = []
    page_token = None
    pbar = tqdm(desc=f"Páginas: {calendar_id}", unit="page", leave=False)

    while True:
        res = service.events().list(
            calendarId=calendar_id,
            timeMin=time_min,
            timeMax=time_max,
            singleEvents=True,
            orderBy="startTime",
            maxResults=max_results,
            timeZone=TZ_DEFAULT,
            fields="items(summary,description,colorId,start(date,dateTime)),nextPageToken",
            pageToken=page_token
        ).execute()

        items = res.get("items", [])
        events.extend(items)
        pbar.update(1)

        page_token = res.get("nextPageToken")
        if not page_token:
            break

    pbar.close()
    return events

## 8) Descargar eventos de los 3 calendarios y construir DataFrame

In [ ]:
rows = []
local_tz = pytz.timezone(TZ_DEFAULT)

for cid in tqdm(CAL_IDS, desc="Calendarios", unit="cal"):
    events = fetch_events(service, cid, start_iso, end_iso)

    cal_list = service.calendarList().get(calendarId=cid).execute()
    cal_summary = cal_list.get('summary', cid)

    for ev in tqdm(events, desc=f"Eventos: {cal_summary}", unit="evt", leave=False):
        start_info = ev.get('start', {})
        start_raw_date = start_info.get('date')
        start_raw_dt   = start_info.get('dateTime')

        # Fecha sola (para día completo o con hora)
        try:
            start_date = pd.to_datetime(start_raw_date or start_raw_dt).date()
        except Exception:
            start_date = None

        # Fecha + hora en TZ_LOCAL
        try:
            if start_raw_dt:
                # Tiene hora → parsear y convertir a TZ_DEFAULT
                dt_parsed = pd.to_datetime(start_raw_dt)
                if dt_parsed.tzinfo is None:
                    dt_parsed = dt_parsed.tz_localize("UTC")
                start_datetime = dt_parsed.astimezone(local_tz)
            elif start_raw_date:
                # Día completo → poner hora 00:00:00 en TZ_DEFAULT
                dt_parsed = pd.to_datetime(start_raw_date)
                start_datetime = local_tz.localize(dt_parsed)
            else:
                start_datetime = None
        except Exception:
            start_datetime = None

        rows.append({
            "calendar_id": cid,
            "summary": ev.get('summary', ''),
            "description": ev.get('description', ''),
            "color_id": ev.get('colorId', ''),
            "date": start_date,
            "date_time": start_datetime
        })

In [ ]:
df = pd.DataFrame(rows)
df.head()

In [ ]:
keywords = [
    "vacuna", "vacunas", "vac\\.", 
    "clustoid", "clustek", "dosis", "mantención", "mantención",
    "inmunoterapia"
]
pattern = r'(?:' + '|'.join(keywords) + r')'

df['is_vacuna'] = (
    df['summary'].fillna('').str.contains(pattern, flags=re.IGNORECASE, regex=True) |
    df['description'].fillna('').str.contains(pattern, flags=re.IGNORECASE, regex=True)
)

# Patrón para detectar COVID y sus variantes
covid_pattern = r'covid(?:[\s-]*19)?'

df['is_covid'] = (
    df['summary'].fillna('').str.contains(covid_pattern, flags=re.IGNORECASE, regex=True) |
    df['description'].fillna('').str.contains(covid_pattern, flags=re.IGNORECASE, regex=True)
)

In [ ]:
df_summary = (
    df.assign(
        is_vacuna=df['is_vacuna'].fillna(False),
        is_covid=df['is_covid'].fillna(False),
        no_vac_no_covid=lambda x: (~x['is_vacuna'] & ~x['is_covid']).astype(int),
        no_vac_si_covid=lambda x: (~x['is_vacuna'] &  x['is_covid']).astype(int),
        si_vac_no_covid=lambda x: ( x['is_vacuna'] & ~x['is_covid']).astype(int),
        si_vac_si_covid=lambda x: ( x['is_vacuna'] &  x['is_covid']).astype(int),
    )
    .groupby('color_id')
    .agg(
        first_date=('date', 'min'),
        count=('date', 'size'),
        last_date=('date', 'max'),
        vacuna_count=('is_vacuna', 'sum'),
        covid_count=('is_covid', 'sum'),
        no_vac_no_covid=('no_vac_no_covid', 'sum'),
        no_vac_si_covid=('no_vac_si_covid', 'sum'),
        si_vac_no_covid=('si_vac_no_covid', 'sum'),
        si_vac_si_covid=('si_vac_si_covid', 'sum'),
    )
    .reset_index()
    .sort_values('first_date')
)

In [ ]:
df_summary

In [ ]:
df_vacuna = df[df['is_vacuna']].copy()

In [ ]:
data = df_vacuna.copy()
data['date'] = pd.to_datetime(data['date'])  # aquí sí conviertes

In [ ]:
# === 1) Campos de fecha útiles ===
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['month_name'] = data['date'].dt.month_name()
data['weekday'] = data['date'].dt.weekday
data['weekday_name'] = data['date'].dt.day_name()

In [ ]:
# Orden y etiquetas (lunes a sábado)
months_order = list(range(1, 13))
weekday_order = list(range(0, 6))  # 0=Lun ... 5=Sáb
weekday_labels = ['Lun','Mar','Mié','Jue','Vie','Sáb']
month_labels = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep','Oct','Nov','Dic']
colors = ['#4C72B0', '#55A868', '#C44E52']  # Azul, verde, rojo

In [ ]:
from datetime import datetime

fecha_hoy = datetime.today().strftime('%d-%m-%Y')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 9), layout='tight')

# 1) Por año
counts_year = data['year'].value_counts().sort_index()
counts_year.plot(kind='bar', ax=axes[0,0], color=colors[0], rot=45)
axes[0,0].set_title('Tratamientos por año')
axes[0,0].set_xlabel('Año')
axes[0,0].set_ylabel('Conteo')

# 2) Por mes (agregado)
counts_month = data['month'].value_counts().reindex(months_order, fill_value=0)
counts_month.plot(kind='bar', ax=axes[0,1], color=colors[1])
axes[0,1].set_title('Tratamientos por mes')
axes[0,1].set_xlabel('Mes')
axes[0,1].set_ylabel('Conteo')
axes[0,1].set_xticks(range(len(month_labels)))
axes[0,1].set_xticklabels(month_labels, rotation=0)

# 3) Por día de la semana (lunes a sábado)
counts_weekday = (
    data.loc[data['weekday'] <= 5, 'weekday']
    .value_counts().reindex(weekday_order, fill_value=0)
)
counts_weekday.plot(kind='bar', ax=axes[1,0], color=colors[2])
axes[1,0].set_title('Tratamientos por día de la semana')
axes[1,0].set_xlabel('Día de la semana')
axes[1,0].set_ylabel('Conteo')
axes[1,0].set_xticks(range(len(weekday_labels)))
axes[1,0].set_xticklabels(weekday_labels, rotation=0)

# 4) Heatmap Año × Mes
piv = (
    data.pivot_table(index='year', columns='month', values='date',
                     aggfunc='size', fill_value=0)
    .reindex(columns=months_order)
)
im = axes[1,1].imshow(piv.values, aspect='auto', cmap='coolwarm')

axes[1,1].set_title('Tratamientos por año y mes')
axes[1,1].set_xlabel('Mes')
axes[1,1].set_ylabel('Año')
axes[1,1].set_xticks(range(len(month_labels)))
axes[1,1].set_xticklabels(month_labels, rotation=0)
axes[1,1].set_yticks(range(len(piv.index)))
axes[1,1].set_yticklabels(piv.index.astype(str))

# Anotaciones en cada celda del heatmap
vals = piv.values
for i in range(vals.shape[0]):
    for j in range(vals.shape[1]):
        axes[1,1].text(
            j, i, str(int(vals[i, j])),
            ha='center', va='center',
            fontsize=10,
            color='white' if vals[i, j] > vals.max() * 0.6 else 'black'
        )

# Barra de color para el heatmap
cbar = fig.colorbar(im, ax=axes[1,1], fraction=0.046, pad=0.04)
cbar.set_label('Conteo', rotation=90)

# Anotar valores sobre las barras
for ax in axes.flatten():
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom', fontsize=10)
    ax.title.set_fontsize(18)
    ax.title.set_weight('bold')
fig.text(0.001, 0.001, 'Bioalergia Chile | www.bioalergia.cl | ' + fecha_hoy,
         fontsize=10, color='gray', ha='left')
logo = mpimg.imread('logo.png')
fig.figimage(logo, alpha=0.1, zorder=-1)
plt.savefig('resumen_general.pdf',dpi=1200)
plt.show()

In [ ]:
# Datos 2025
data_2025 = (
    data.loc[(data['year'] == 2025) & (data['date'].dt.month <= 9)].copy()
    .assign(
        month=lambda d: d['date'].dt.month,
        weekday=lambda d: d['date'].dt.weekday,
        wom=lambda d: ((d['date'].dt.day - 1) // 7) + 1
    )
)

month_labels  = ['Ene','Feb','Mar','Abr','May','Jun','Jul','Ago','Sep']
months_order  = list(range(1, 10))
weekday_labels = ['Lun','Mar','Mié','Jue','Vie','Sáb']
weekday_order  = [0,1,2,3,4,5]

# paleta simple para los 4 paneles (sin forzar colores exactos)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

fig, axes = plt.subplots(2, 2, figsize=(15, 10), layout='tight')

# === 1) Eventos por mes (2025) ===
counts_month = data_2025['month'].value_counts().reindex(months_order, fill_value=0)
counts_month.plot(kind='bar', ax=axes[0,0], color=colors[0])
axes[0,0].set_title('Eventos por mes (2025)')
axes[0,0].set_xlabel('Mes'); axes[0,0].set_ylabel('Conteo')
axes[0,0].set_xticks(range(len(month_labels)))
axes[0,0].set_xticklabels(month_labels, rotation=0)

# === 2) Eventos por día de la semana (Lun–Sáb) ===
counts_weekday = (
    data_2025.loc[data_2025['weekday'] <= 5, 'weekday']
    .value_counts().reindex(weekday_order, fill_value=0)
)
counts_weekday.plot(kind='bar', ax=axes[0,1], color=colors[1])
axes[0,1].set_title('Eventos por día de la semana (2025)')
axes[0,1].set_xlabel('Día de la semana'); axes[0,1].set_ylabel('Conteo')
axes[0,1].set_xticks(range(len(weekday_labels)))
axes[0,1].set_xticklabels(weekday_labels, rotation=0)

# === 3) Día de la semana en cada mes (stacked) ===
pivot_month_weekday = (
    data_2025.loc[data_2025['weekday'] <= 5]
    .pivot_table(index='month', columns='weekday', values='date', aggfunc='size', fill_value=0)
    .reindex(index=months_order, columns=weekday_order)
)
pivot_month_weekday.plot(kind='bar', stacked=True, ax=axes[1,0])
axes[1,0].set_title('Eventos por día de la semana en cada mes (2025)')
axes[1,0].set_xlabel('Mes'); axes[1,0].set_ylabel('Conteo')
axes[1,0].legend(weekday_labels, ncol=3, frameon=False)
axes[1,0].set_xticks(range(len(month_labels)))
axes[1,0].set_xticklabels(month_labels, rotation=0)

d25 = data_2025.copy()
month_start = d25['date'].values.astype('datetime64[M]')
d25['week_of_month'] = ((d25['date'] - month_start).dt.days // 7 + 1).clip(1, 5)

piv_wom = (d25
           .groupby(['month','week_of_month'])
           .size()
           .unstack(fill_value=0)
           .reindex(index=months_order, columns=[1,2,3,4,5]))

piv_wom.plot(kind='bar', ax=axes[1,1], width=0.85)

axes[1,1].set_title('Eventos por semana del mes (2025)')
axes[1,1].set_xlabel('Mes')
axes[1,1].set_ylabel('Conteo')
axes[1,1].set_xticklabels(month_labels, rotation=0)
axes[1,1].legend([f'Sem {i}' for i in [1,2,3,4,5]], fontsize=12)

# — Toques opcionales de estética y valores encima de las barras —
for ax in [axes[0,0], axes[0,1]]:
    for p in ax.patches:
        h = p.get_height()
        if h > 0:
            ax.annotate(f'{int(h)}',
                        (p.get_x() + p.get_width() / 2, h),
                        ha='center', va='bottom', fontsize=10)

# Totales solo para el stacked (3er gráfico)
totales_mes = pivot_month_weekday.sum(axis=1).values
for i, total in enumerate(totales_mes):
    axes[1,0].annotate(f'{int(total)}',
                       (i, total),
                       ha='center', va='bottom', fontsize=10)
for ax in axes.flatten():
    ax.title.set_fontsize(18)
    ax.title.set_weight('bold')
    ax.xaxis.label.set_fontsize(14)
    ax.yaxis.label.set_fontsize(14)
    ax.tick_params(axis='both', labelsize=14)

fig.text(0.001, 0.001, 'Bioalergia Chile | www.bioalergia.cl | ' + fecha_hoy,
         fontsize=10, color='gray', ha='left')
logo = mpimg.imread('logo.png')
fig.figimage(logo, alpha=0.1, zorder=-1)
plt.savefig('resumen_2025.pdf',dpi=1200)
plt.show()

In [ ]:
def calendar_heatmap_month(df, year, month,
                           monday_to_saturday=True,
                           cmap='YlOrRd',
                           ax=None,
                           vmin=None, vmax=None,
                           show_week_sums=True):
    """
    Heatmap tipo calendario para (year, month).
    Si show_week_sums=True agrega una columna 'Σ' con el total por semana.
    Devuelve (fig, ax, im).
    """
    # --- datos del mes ---
    d = df[(df['date'].dt.year == year) & (df['date'].dt.month == month)]
    counts = d.groupby(d['date'].dt.day).size()

    cal = calendar.Calendar(firstweekday=0)  # 0=Lunes
    weeks = cal.monthdayscalendar(year, month)  # lista de semanas (7 columnas con 0s)

    if monday_to_saturday:
        weeks = [w[:-1] for w in weeks]  # recorta domingo
        ncols = 6
        weekday_labels = ['Lun','Mar','Mié','Jue','Vie','Sáb']
    else:
        ncols = 7
        weekday_labels = ['Lun','Mar','Mié','Jue','Vie','Sáb','Dom']

    nrows = len(weeks)
    grid = np.zeros((nrows, ncols), dtype=int)
    mask_empty = np.zeros((nrows, ncols), dtype=bool)

    for i, w in enumerate(weeks):
        for j, day in enumerate(w):
            if day == 0:
                mask_empty[i, j] = True
            else:
                grid[i, j] = int(counts.get(day, 0))
    # --- figura/ejes ---
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 3.8), constrained_layout=True)
    else:
        fig = ax.figure

    # color scale común si se pasa vmax
    im = ax.imshow(grid, cmap=cmap, aspect='equal', vmin=vmin, vmax=vmax, interpolation='nearest')

    # celdas vacías en gris claro (debajo del heatmap)
    for i in range(nrows):
        for j in range(ncols):
            if mask_empty[i, j]:
                ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1,
                                           facecolor='#f0f0f0', edgecolor='none', zorder=-1))

    # anotaciones: día (arriba-izq) y conteo (centrado)
    gmax = grid.max() if grid.size else 0
    for i in range(nrows):
        for j in range(ncols):
            if not mask_empty[i, j]:
                day = weeks[i][j]
                ax.text(j-0.43, i-0.4, str(day), fontsize=8, ha='left', va='top', color='black')
                ax.text(j, i, str(grid[i, j]), fontsize=10, ha='center', va='center',
                        color=('white' if gmax and grid[i, j] > gmax*0.6 else 'black'),fontweight='bold')

    today = pd.Timestamp.today().normalize()
    if today.year == year and today.month == month:
        day = today.day
        for i, w in enumerate(weeks):
            for j, dday in enumerate(w if not monday_to_saturday else w):
                if dday == day:
                    circ = plt.Circle((j, i), 0.2, fill=False, linewidth=2, color='tab:blue')
                    ax.add_patch(circ)
    # ejes
    ax.set_xticks(range(ncols))
    ax.set_xticklabels(weekday_labels)
    ax.set_yticks(range(nrows))
    ax.set_yticklabels([f'Sem {k+1}' for k in range(nrows)])
    ax.set_title(f"{calendar.month_name[month]} {year} | Σ={int(grid.sum())}",
             fontweight='bold')
    ax.set_xlabel('Día de la semana'); ax.set_ylabel('Semanas')

    fig.text(0.001, 0.001, 'Bioalergia Chile | www.bioalergia.cl | ' + fecha_hoy,
         fontsize=10, color='gray', ha='left')
    # bordes finos y sin grid
    for spine in ax.spines.values():
        spine.set_visible(False)

    # --- columna con sumas por semana ---
    if show_week_sums:
        week_sums = grid.sum(axis=1)
        # ampliamos el eje a una columna extra
        ax.set_xlim(-0.5, ncols + 0.6)
        # “columna Σ” en gris suave
        for i in range(nrows):
            ax.add_patch(plt.Rectangle((ncols-0.5, i-0.5), 1.0, 1.0,
                                       facecolor='#efefef', edgecolor='white'))
            ax.text(ncols, i, str(int(week_sums[i])), ha='center', va='center',
                    fontsize=10, color='black', fontweight='bold')
        # etiqueta Σ
        ax.text(ncols, -0.9, 'Total', ha='center', va='center', fontsize=11, fontweight='bold')
        # ticks: añadimos un tick “visual” al final (sin etiqueta en el eje)
        ax.set_xticks(list(range(ncols)) + [ncols])
        ax.set_xticklabels(weekday_labels + [''])
    return fig, ax, im

In [ ]:
months = [7,8, 9]  # Ago, Sep, Oct
# mismo máximo para todos (solo cuentas de estos meses)
vmax_global = 0
for m in months:
    v = (df_vacuna
         .loc[(data['date'].dt.year==2025) & (data['date'].dt.month==m)]
         .groupby(data['date'].dt.day).size()
         .max())
    vmax_global = max(vmax_global, int(v or 0))

fig, axes = plt.subplots(1, 3, figsize=(16, 4.5), layout='constrained')
for ax, m in zip(axes, months):
    calendar_heatmap_month(
        data, 2025, m,
        monday_to_saturday=True,
        cmap='YlOrRd',
        ax=ax,
        show_week_sums=True              # << suma por semana
    )
plt.savefig('8910_15-08_vacunas.pdf',dpi=300)
plt.show()

In [3]:
import imaplib
import email
from email.header import decode_header, make_header
import os

def decode_str_header(hdr):
    # hdr puede ser None
    if hdr is None:
        return ""
    # make_header + str convierte un objeto Header en str decodificado
    return str(make_header(decode_header(hdr)))

def export_folder(imap, folder, out_dir):
    status, _ = imap.select(f'"{folder}"', readonly=True)
    if status != "OK":
        print("Error al abrir carpeta", folder)
        return
    status, data = imap.search(None, "ALL")
    if status != "OK":
        print("Error al buscar correos en", folder)
        return
    uids = data[0].split()
    os.makedirs(out_dir, exist_ok=True)
    for uid in uids:
        status, msg_data = imap.fetch(uid, "(RFC822)")
        if status != "OK":
            print("Error al fetch", uid)
            continue
        raw = msg_data[0][1]
        msg = email.message_from_bytes(raw)
        subj = decode_str_header(msg.get("Subject"))
        # limpiar nombre de archivo
        safe_subj = subj.replace("/", "_").replace("\\", "_")
        # UID es bytes, decodificar
        uid_str = uid.decode() if isinstance(uid, bytes) else str(uid)
        filename = f"{uid_str}_{safe_subj}.eml"
        path = os.path.join(out_dir, filename)
        with open(path, "wb") as f:
            f.write(raw)

def export_all(imap_host, username, password, base_out="backup"):
    imap = imaplib.IMAP4_SSL(imap_host)
    imap.login(username, password)
    status, folders = imap.list()
    if status != "OK":
        print("No se pudo listar carpetas")
        return
    for folder_entry in folders:
        decoded = folder_entry.decode()
        # parsing de línea tipo: b'(\\HasNoChildren) "/" "INBOX"' etc
        parts = decoded.split(' "/" ')
        if len(parts) == 2:
            folder = parts[1].strip('"')
        else:
            folder = decoded.split()[-1].strip('"')
        safe = folder.replace("/", "_")
        out_dir = os.path.join(base_out, safe)
        export_folder(imap, folder, out_dir)
    imap.logout()

if __name__ == "__main__":
    HOST = "imap.titan.email"
    USER = "tuusuario@tudominio"
    PASS = "tucontraseña"
    export_all('imap.titan.email', 'contacto@bioalergia.cl', 'Bioalergia870.', base_out="titan_backup/contacto")